## Operando XRD

In [ ]:
# -*- coding: utf-8 -*-
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import dates, gridspec, ticker, transforms
from matplotlib.colorbar import Colorbar
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# xarray setting
xr.set_options(
    cmap_sequential="viridis",
    cmap_divergent="viridis",
    display_width=150,
)  # viridis, gray

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#b0a3d1", "#8bd0d5", "#a8e0ee", "#c5e1a3", "#ffe48b", "#f5a37d", "#e88db1"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

### Operando XRD, NOTOs-2023

##### 谱线上的时间

In [ ]:
# 读取文件中时间戳
filelist = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2023-NOTOs\Results\Overview\IS13MnO2Znpd"
    ).glob(r"*.xye")
)
range_index, wave_length, time_processed = [], [], []

for path_file in filelist:
    with open(path_file, "r") as file:
        lines = file.readlines()
        file_id = path_file.stem.split("-")[-1].split(".")[0][-5:]
        range_index.append(file_id)

        # 解析文件内容
        for line in lines:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
                wave_length.append(wave_value)
            # elif line.startswith('# Date'):
            #     recored_time = str(line.split()[3])
            #     time_processed.append(recored_time)

df = pd.DataFrame({
    "Range_Index": range_index,
    # 'Time': time_processed,
    "Wave_Length": wave_length,
})
# df['Time'] = pd.to_datetime(df['Time'],format=r'%Y-%m-%d_%H:%M:%S')
df["Range_Index"] = pd.to_numeric(df["Range_Index"])
df["Wave_Length"] = pd.to_numeric(df["Wave_Length"])
df.to_csv(
    Path.joinpath(path_out, "Time_index_spectrum.csv"),
    sep=",",
    header=True,
    columns=["Range_Index", "Wave_Length"],
    index=False,
)
# df.to_csv(path.joinpath(path_out, 'Time_index_spectrum.csv'), sep=',', header=True, columns=['Range_Index', 'Time', 'Wave_Length'], index=False,)

##### 匹配谱线和电化学上的时间

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2023-NOTOs\Results\Overview"
)
echem_file = Path.joinpath(path_file, r"Echem", r"13MnO2_C03.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"IS13MnO2Znpd", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
)
# time_spectrum.info()

# 匹配谱线和电化学上的时间
time_spectrum_index = np.linspace(0, echem.shape[0], time_spectrum.shape[0])
index_spectrum = [np.abs(echem.index - t).argmin() for t in time_spectrum_index]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
# # index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage")

selected_times = echem["time/s"].iloc[index_spectrum]
selected_voltages = echem["Ewe/V"].iloc[index_spectrum]
ax.scatter(selected_times, selected_voltages, c=colors[1])

# 添加索引文本标注
row_index = index_voltage["index"].loc[index_voltage.index.isin([0,4,6,8,10,13,
                                                                 15,18,20,22,24,26,29,32,53,55,58,61,62,65,67,70,71,74,94,])]
for i, idx in enumerate(row_index):
    ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], marker="o")
    ax.text(
        echem["time/s"].iloc[idx],
        echem["Ewe/V"].iloc[idx] + 0.03,
        str(row_index.index[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max())
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", labelsize=9, right=True, labelright=True)

ax.legend(loc="upper left", bbox_to_anchor=(0.0, 1.05), frameon=False, fontsize=11)
ax2.legend(loc="upper left", bbox_to_anchor=(0.2, 1.05), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_NOTOs_1_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_NOTOs_1_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_NOTOs_1_600_echem.png"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

##### Overview

In [ ]:
# 读取 alfaMnO2capillary 数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2023-NOTOs\Results")
ref_xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Exsitu", r"Capillaries", r"spectrum.csv"), sep=",", index_col=None, header=0, comment="#"
)
# 读取 PDF card 的数据
pdf = pd.read_excel(
    Path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_1_12750.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
pdf = pdf[(pdf["Intensity"] >= 10) & (pdf["2θ"] <= 45) & (pdf["2θ"] >= 10)]

In [ ]:
def extract_wave_and_data(file_path):
    wave_value = None
    recored_time = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
            elif line.startswith("# Date"):
                recored_time = line.split()[3]
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recored_time, wave_value, data


def process_files(directory):
    results = []

    for file_path in Path(directory).glob("*.xye"):
        recored_time, wave_value, data = extract_wave_and_data(file_path)
        file_id = file_path.stem.split("_")[-1][-5:]  # Extract the last part of the filename
        for row in data:
            results.append([float(file_id), recored_time, wave_value] + row)

    return results


# 读取 XRD 数据，并清洗
directory = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2023-NOTOs\Results\Overview\IS13MnO2Znpd"
)
spectum_all = process_files(directory)

# 转换为 DataFrame 并进行数据处理
spectum_all = pd.DataFrame(
    spectum_all, columns=["range", "recored_time", "wave_value", "2THETA", "Intensity1", "Intensity2"]
)
spectum_all["Cnt2_D1"] = spectum_all["Intensity1"] + spectum_all["Intensity2"]
spectum_all = spectum_all[["range", "2THETA", "Cnt2_D1"]]

In [ ]:
spectum_all = pd.concat(
    [s.set_index("2THETA")["Cnt2_D1"].rename(i) for i, s in spectum_all.groupby("range")],
    axis=1
).reset_index(drop=False)
spectum_all

In [ ]:
%matplotlib inline
Letters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 5.0))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2], height_ratios=None, wspace=0, hspace=0, figure=fig)
values = [0,4,13,22,32,62,74]
# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(1.0)

for i, col in enumerate(spectum_all.columns[1:], start=0):
    ax.plot(
        spectum_all.iloc[:, 0],
        spectum_all[col] + 2000 * i,
        ls="-", lw=1.0,
        c='grey' if float(col) not in values else colors[3],
        label=None, alpha=0.4 if float(col) not in values else 1.0, zorder=0 if float(col) not in values else 5,
    )

ax.set_ylabel(
    r"Intensity (arb.u.)",
    fontsize=11,
)
# ax.set_ylim(6000, 350000)

ax.set_xlabel(
    r"2Theta ($ \mathrm{\theta}$)",
    fontsize=11,
)
# ax.set_xlim(4, 24)
# ax.xaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=0))
# ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=0))
ax.tick_params(axis="both", which="both", labelsize=9, top=False, right=False, left=False, labelleft=False)


plt.savefig(
    Path.joinpath(path_out, r"opXRD_MSDP_XRD_01_300.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
# spectum_all.drop(spectum_all[spectum_all['range'] == 28].index, axis=0, inplace=True)

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2023-NOTOs\Results\Overview"
)
echem_file = Path.joinpath(path_file, r"Echem", r"13MnO2_C03.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"IS13MnO2Znpd", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
)
# time_spectrum.info()

# 匹配谱线和电化学上的时间
time_spectrum_index = np.linspace(0, echem.shape[0], time_spectrum.shape[0])
index_spectrum = [np.abs(echem.index - t).argmin() for t in time_spectrum_index]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [0, 4, 6, 8, 10, 13, 15, 18, 20, 22, 24, 26, 29, 32, 53, 55, 58, 61, 62, 65, 67, 70, 71, 74, 94]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all.iloc[:, 0] <= selected["level_0"].iloc[-1] + 1].pivot(
    index=["2THETA"], columns=["range"], values=["Cnt2_D1"]
)

In [ ]:
def plot_echem_voltage_time(ax, selected_echem, selected, colors, index):
    """绘制电化学，电压-时间图"""
    ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoinA")
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
    )
    # 特殊位置颜色标定
    for idx in index:
        temp = selected["index"][selected["level_0"] == idx]
        ax.scatter(selected_echem["Ewe/V"].iloc[temp], selected_echem["time/s"].iloc[temp], c=colors[0], marker="o")

    # 添加索引文本标注
    for i in range(selected.shape[0]):
        temp = selected["index"].iloc[i]
        ax.text(
            selected_echem["Ewe/V"].iloc[temp] + 0.15,
            selected_echem["time/s"].iloc[temp],
            str(selected["level_0"].iloc[i]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

    ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
    ax.set_xlim(0.8, 2.0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

    # 确保时间刻度从数据最开始时间显示
    ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
    ax.set_ylim(
        selected_echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max() + pd.Timedelta(minutes=20)
    )
    ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
    ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
    ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))
    plt.yticks(rotation=0, horizontalalignment="right")

    ax.text(
        -0.2,
        -0.06,
        r"$\mathrm{2.128 mg\ cm^{-2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )
    ax.text(
        -0.15,
        0,
        r"C/10, 1C=300 mA/g",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )


def plot_current(ax2, selected_echem, colors):
    ax2.plot(selected_echem["<I>/mA"], selected_echem["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")
    ax2.set_xlabel(
        r"Current (mA)",
        fontsize=11,
    )
    ax2.set_xlim(-0.1, 0.1)
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
    ax2.tick_params(axis="both", which="both", labelsize=9, top=True, labeltop=True)
    # ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)


def plot_xrd_spectra(ax, index, spectum_all, selected, colors, ref_xrd, pdfcard, offset=300):
    """绘制 XRD 光谱图"""
    xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
    for i in range(selected.shape[0]):
        ax.plot(
            spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
            spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + offset * (i + 1),
            ls="-",
            lw=1.0,
            c=xrd_colors[i],
            label=str(selected["level_0"].iloc[i]),
        )

    #   标样和 pdf 卡片
    ax.plot(
        ref_xrd.iloc[:, 0], 0.1 * (ref_xrd.iloc[:, 1] + 4000), ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$"
    )
    ax.stem(
        pdfcard.iloc[:, 0],
        pdfcard.iloc[:, 2] * 8,
        linefmt=colors[3],
        markerfmt="none",
        bottom=0,
        orientation="vertical",
        label=r"PDF #00-044-0141",
    )
    ax.text(
        0.95,
        0.95,
        r"PDF + Exp.${\mathrm{\alpha\!-\!MnO_2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )

    ax.set_xlim(10, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

    ax.set_ylim(0, 27000)
    ax.set_ylabel("Relative Intensity (arb.u.)", fontsize=11)
    ax.set(yticks=[])


def plot_contour_map(ax, mapping_spectum, cmap="coolwarm"):
    """绘制等高线图"""
    ax.contourf(
        mapping_spectum.index, mapping_spectum.columns.to_frame()["range"], mapping_spectum.T, cmap=cmap, levels=100
    )
    ax.set_xlim(10, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))
    ax.set_yticks([])


def add_colorbar(ax, cmap="coolwarm"):
    """添加颜色条"""
    colorbar = Colorbar(
        ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
        location="right",
        orientation="vertical",
        cmap=cmap,
        ticklocation="right",
        spacing="proportional",
        drawedges=False,
    )
    colorbar.set_ticks([])
    ax.text(1.08, 0.97, "High", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")
    ax.text(1.08, 0.08, "Low", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")


def add_pdf(ax, pdfcard):
    scale_pdf = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
    offst_pdf = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
    XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
    for i in range(pdfcard.shape[1] // 3):
        temp = pdfcard.iloc[:, [3 * i, 3 * i + 2]].dropna(axis="index", how="all")
        for j in range((temp.shape[0])):
            ax.axvline(
                x=temp.iloc[j, 0],
                ymin=offst_pdf[i],
                ymax=(offst_pdf[i] + temp.iloc[j, 1] * scale_pdf[i]),
                lw=1,
                c=XRDcolors[i],
            )


def add_box(subfig):
    ax = subfig.add_subplot()
    ax.set_position([0, -0.25, 0.6, 0.15])
    ax.text(
        0,
        0.05,
        r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
        horizontalalignment="left",
        verticalalignment="bottom",
        transform=ax.transAxes,
        fontsize=10,
        c="blue",
    )
    ax.spines[:].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor("gainsboro")


def plot_reference_peaks(subfig, mapping_spectum, ref_peaks, colors, index, position=0.5):
    """绘制参考峰"""
    for i, peak in enumerate(ref_peaks):
        ax = subfig.add_axes((0 + i * 0.07, position, 0.06, 0.8), zorder=0)
        temp = mapping_spectum.loc[(mapping_spectum.index >= peak[0] - 0.1) & (mapping_spectum.index <= peak[0] + 0.1)]
        ax.contourf(temp.index, temp.columns.to_frame()["range"], temp.T, cmap="coolwarm", levels=100)
        ax.axvline(peak[0], ls="--", lw=1.0, color="k", alpha=0.6)
        ax.set_xlim(temp.index.min(), temp.index.max())
        ax.yaxis.set_ticks([])
        ax.xaxis.set_major_formatter(ticker.FormatStrFormatter("%.2f"))
        ax.xaxis.set_major_locator(ticker.FixedLocator(peak))
        ax.spines[["top", "right", "left"]].set_visible(False)
        ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)  # colors[i]
        for j in index:
            ax.axhline(y=j + 1, lw=0.6, ls="--")


# 绘图
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

specical_index = [0, 4, 8, 13, 18, 22, 26, 29, 32, 53, 58, 62, 67, 70, 74, 94]
plot_echem_voltage_time(ax, selected_echem, selected, colors, specical_index)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)
plot_current(ax2, selected_echem, colors)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

plot_xrd_spectra(ax, specical_index, spectum_all, selected, colors, ref_xrd, pdf, offset=800)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)
plot_contour_map(ax, mapping_spectum)
add_colorbar(ax)
add_pdf(ax, pdf)

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
add_box(subfig)
opXRD_ref_peak = pdf.iloc[:, 0].apply(lambda x: [x]).to_list()
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]
plot_reference_peaks(subfig, mapping_spectum, opXRD_ref_peak, opXRD_colors, specical_index, position=0.0)

# 保存图像
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_opData_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_opData_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_opData_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

### Operando XRD, NOTOs-2024

#### Ex situ, WL=0.9535

##### Align Data

In [ ]:
import csv
from pathlib import Path

import pandas as pd


def extract_wave_and_data(file_path):
    """提取波长、记录时间和数据"""
    wave_length = None
    recorded_time = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_length = float(line.split()[3])
            elif line.startswith("# Date"):
                recorded_time = line.split()[3]
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recorded_time, wave_length, data


def process_files(directory):
    """处理目录中的所有 .xye 文件"""
    directory = Path(directory)
    results = []

    for file_path in directory.glob("*.xye"):
        recorded_time, wave_length, data = extract_wave_and_data(file_path)
        file_id = file_path.stem  # 文件名去除后缀
        results.extend([[file_id, recorded_time, wave_length] + row for row in data])

    return results


# 读取 XRD 数据，并保存为 CSV
directory = Path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Exsitu\Align"
)
results = process_files(directory)
columns = ["Number", "Recorded_time", "WL", "2THETA", "Intensity1", "Intensity2"]

pd.DataFrame(results, columns=columns).to_csv(path.joinpath(path_out, f"{directory.stem}.csv"), index=False)

In [ ]:
# 读取数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results")
pdf = pd.read_excel(
    path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_0_9535.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Exsitu", r"Align", r"Align.csv"), sep=",", index_col=None, header=0, comment="#"
)
xrd["sum"] = xrd["Intensity1"] + xrd["Intensity2"]
xrda = xrd.pivot(index="2THETA", columns="Number", values="sum").reset_index(drop=False)
xrda.to_csv(path.joinpath(path_out, "spectrum.csv"), header=True, index=None)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)
labels = [r"$\alpha$-MnO${_2}$", None, None, None]
for i in range(xrda.shape[1] - 9):
    temp = xrda.iloc[:, [0, i + 7]].dropna(axis=0, how="any").iloc[1:, :]
    ax.plot(temp.iloc[:, 0], temp.iloc[:, 1] + 500 * i, ls="-", lw=1.0, c=colors[i], label=labels[i])

ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 10 + 0,
    linefmt=colors[3],
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(10, 50)
ax.set_xlabel(
    "2-Theta (degree), $\mathrm{\lambda = 0.953 \AA}$",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=-5))

ax.set_ylim(0, 7000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.90,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_1_alfaMnO2Align_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_1_alfaMnO2Align_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_1_alfaMnO2Align_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)
labels = [r"ZHS", None, None, None, None, None]
for i in range(xrda.shape[1] - 5):
    temp = xrda.iloc[:, [0, i + 1]].dropna(axis=0, how="any").iloc[1:, :]
    ax.plot(temp.iloc[:, 0], temp.iloc[:, 1] + 1000 * i, ls="-", lw=1.0, c=colors[i], label=labels[i])

ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

ax.set_xlim(10, 50)
ax.set_xlabel(
    "2-Theta (degree), $\mathrm{\lambda = 0.953 \AA}$",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=-5))

ax.set_ylim(1000, 14000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_2_alfaMnO2Align_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_2_alfaMnO2Align_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_2_alfaMnO2Align_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)
labels = [r"opCell", None]
for i in range(xrda.shape[1] - 9):
    temp = xrda.iloc[:, [0, i + 9]].dropna(axis=0, how="any").iloc[1:, :]
    ax.plot(temp.iloc[:, 0], temp.iloc[:, 1] + 500 * i, ls="-", lw=1.0, c=colors[i], label=labels[i])

ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 10 + 500,
    linefmt=colors[3],
    markerfmt="none",
    bottom=500,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(10, 50)
ax.set_xlabel(
    "2-Theta (degree), $\mathrm{\lambda = 0.953 \AA}$",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=-5))

ax.set_ylim(500, 7000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.90,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_3_alfaMnO2Align_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_3_alfaMnO2Align_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_3_alfaMnO2Align_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

##### References

In [ ]:
import csv
import os


def extract_wave_and_data(file_path):
    wave_value = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
            if line.startswith("# Date"):
                recored_time = str(line.split()[3])
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recored_time, wave_value, data


def process_files(directory):
    results = []

    for filename in os.listdir(directory):
        if filename.endswith(".xye"):
            file_path = os.path.join(directory, filename)
            recored_time, wave_value, data = extract_wave_and_data(file_path)
            file_id = filename.split(".")[0]  # Extract the last part of the filename
            for row in data:
                results.append([file_id, recored_time, wave_value] + row)

    return results


# 读取 XRD 数据，并清洗
directory = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Exsitu\References"
)
results = process_files(directory)
pd.DataFrame(results).to_csv(
    path.joinpath(path_out, f"{directory.stem}.csv"),
    header=[r"Number", r"Recorded_time", r"WL", r"2THETA", r"Intensity1", r"Intensity2"],
    index=None,
)

In [ ]:
# 读取数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results")
pdf = pd.read_excel(
    path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_0_9535.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Exsitu", r"References", r"References.csv"), sep=",", index_col=None, header=0, comment="#"
)
xrd["sum"] = xrd["Intensity1"] + xrd["Intensity2"]
xrda = xrd.pivot(index="2THETA", columns="Number", values="sum").reset_index(drop=False)
xrda.to_csv(path.joinpath(path_out, "spectrum.csv"), header=True, index=None)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)
labels = [r"ZHS", r"$\mathrm{\alpha MnO_2}$"]
for i in range(xrda.shape[1] - 1):
    temp = xrda.iloc[:, [0, i + 1]].dropna(axis=0, how="any").iloc[1:, :]
    ax.plot(temp.iloc[:, 0], temp.iloc[:, 1] + 2000 * i, ls="-", lw=1.0, c=colors[i], label=labels[i])

ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 50 + 2000,
    linefmt=colors[3],
    markerfmt="none",
    bottom=2000,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(10, 46)
ax.set_xlabel(
    "2-Theta (degree), $\mathrm{\lambda = 0.953 \AA}$",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=-2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=-2))

ax.set_ylim(2000, 15000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.82,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_References_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_References_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_References_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### Operando XRD, CoinB

##### 谱线上的时间

In [ ]:
# 读取文件中时间戳
filelist = list(
    path(
        r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview\opCellB"
    ).glob(r"*.xye")
)
range_index, wave_length, time_processed = [], [], []

for path_file in filelist:
    with open(path_file, "r") as file:
        lines = file.readlines()
        file_id = path_file.stem.split("-")[-1].split(".")[0][-5:]
        range_index.append(file_id)

        # 解析文件内容
        for line in lines:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
                wave_length.append(wave_value)
            elif line.startswith("# Date"):
                recored_time = str(line.split()[3])
                time_processed.append(recored_time)

df = pd.DataFrame({
    "Range_Index": range_index,
    "Time": time_processed,
    "Wave_Length": wave_length,
})
df["Time"] = pd.to_datetime(df["Time"], format=r"%Y-%m-%d_%H:%M:%S")
df["Range_Index"] = pd.to_numeric(df["Range_Index"])
df["Wave_Length"] = pd.to_numeric(df["Wave_Length"])
df.to_csv(
    path.joinpath(path_out, "Time_index_spectrum.csv"),
    sep=",",
    header=True,
    columns=["Range_Index", "Time", "Wave_Length"],
    index=False,
)

##### 匹配谱线和电化学上的时间

In [ ]:
# 电化学上的时间
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview"
)
echem_file = path.joinpath(path_file, r"Echem", r"001-LC-Zn-05MZnSO4-02MMnSO4-alphaMnO2-59mg_02_GCPL_C01.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"opCellB", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum = [np.abs(echem_time - t).argmin() for t in time_spectrum["Time"].values]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
# # index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage")

selected_times = echem["time/s"].iloc[index_spectrum]
selected_voltages = echem["Ewe/V"].iloc[index_spectrum]
ax.scatter(selected_times, selected_voltages, c=colors[1])

# 添加索引文本标注
row_index = index_voltage["index"].iloc[index_voltage.index.isin([0, 3, 8, 25, 30])]
for i, idx in enumerate(row_index):
    ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], marker="o")
    ax.text(
        echem["time/s"].iloc[idx],
        echem["Ewe/V"].iloc[idx] + 0.03,
        str(row_index.index[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max())
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax2 = ax.twinx()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", labelsize=9, right=True, labelright=True)

ax.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)
ax2.legend(loc="upper left", bbox_to_anchor=(0.8, 1), frameon=False, fontsize=11)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

##### Overview

In [ ]:
# 读取 alfaMnO2capillary 数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results")
ref_xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Exsitu", r"References", r"spectrum.csv"), sep=",", index_col=None, header=0, comment="#"
)
# 读取 PDF card 的数据
pdf = pd.read_excel(
    path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_0_9535.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
pdf = pdf[(pdf["Intensity"] >= 10) & (pdf["2θ"] <= 45)]

In [ ]:
from pathlib import Path

import pandas as pd


def extract_wave_and_data(file_path):
    wave_value = None
    recored_time = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
            elif line.startswith("# Date"):
                recored_time = line.split()[3]
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recored_time, wave_value, data


def process_files(directory):
    results = []

    for file_path in Path(directory).glob("*.xye"):
        recored_time, wave_value, data = extract_wave_and_data(file_path)
        file_id = file_path.stem.split("_")[-1][-5:]  # Extract the last part of the filename
        for row in data:
            results.append([float(file_id), recored_time, wave_value] + row)

    return results


# 读取 XRD 数据，并清洗
directory = Path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview\opCellB"
)
spectum_all = process_files(directory)

# 转换为 DataFrame 并进行数据处理
spectum_all = pd.DataFrame(
    spectum_all, columns=["range", "recored_time", "wave_value", "2THETA", "Intensity1", "Intensity2"]
)
spectum_all["Cnt2_D1"] = spectum_all["Intensity1"] + spectum_all["Intensity2"]
spectum_all = spectum_all[["range", "2THETA", "Cnt2_D1"]]

In [ ]:
spectum_all.drop(spectum_all[spectum_all["range"] == 28].index, axis=0, inplace=True)

In [ ]:
# 电化学上的时间
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview"
)
echem_file = path.joinpath(path_file, r"Echem", r"001-LC-Zn-05MZnSO4-02MMnSO4-alphaMnO2-59mg_02_GCPL_C01.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"opCellB", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum = [np.abs(echem_time - t).argmin() for t in time_spectrum["Time"].values]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [0, 3, 5, 8, 10, 12, 14, 17, 21, 25, 30]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all.iloc[:, 0] <= selected["level_0"].iloc[-1] + 1].pivot(
    index=["2THETA"], columns=["range"], values=["Cnt2_D1"]
)

In [ ]:
def plot_echem_voltage_time(ax, selected_echem, selected, colors, index):
    """绘制电化学，电压-时间图"""
    ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoinA")
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
    )
    # 特殊位置颜色标定
    for idx in index:
        temp = selected["index"][selected["level_0"] == idx]
        ax.scatter(selected_echem["Ewe/V"].iloc[temp], selected_echem["time/s"].iloc[temp], c=colors[0], marker="o")

    # 添加索引文本标注
    for i in range(selected.shape[0]):
        temp = selected["index"].iloc[i]
        ax.text(
            selected_echem["Ewe/V"].iloc[temp] + 0.15,
            selected_echem["time/s"].iloc[temp],
            str(selected["level_0"].iloc[i]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

    ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
    ax.set_xlim(0.8, 2.0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

    # 确保时间刻度从数据最开始时间显示
    ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
    ax.set_ylim(
        selected_echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max() + pd.Timedelta(minutes=20)
    )
    ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
    ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
    ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))
    plt.yticks(rotation=0, horizontalalignment="right")

    ax.text(
        -0.2,
        -0.06,
        r"$\mathrm{1.18 mg\ cm^{-2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )
    ax.text(
        -0.15,
        0,
        r"C/2, 1C=300 mA/g",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )


def plot_current(ax2, selected_echem, colors):
    ax2.plot(selected_echem["<I>/mA"], selected_echem["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")
    ax2.set_xlabel(
        r"Current (mA)",
        fontsize=11,
    )
    ax2.set_xlim(-0.1, 0.1)
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
    ax2.tick_params(axis="both", which="both", labelsize=9, top=True, labeltop=True)
    # ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)


def plot_xrd_spectra(ax, index, spectum_all, selected, colors, ref_xrd, pdfcard, offset=300):
    """绘制 XRD 光谱图"""
    xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
    for i in range(selected.shape[0]):
        ax.plot(
            spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
            spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + offset * (i + 1),
            ls="-",
            lw=1.0,
            c=xrd_colors[i],
            label=str(selected["level_0"].iloc[i]),
        )

    #   标样和 pdf 卡片
    ax.plot(ref_xrd.iloc[:, 0], 0.1 * (ref_xrd.iloc[:, 1] + 4000), ls="-", lw=1.0, c=colors[2], label=r"ZHS")
    ax.plot(
        ref_xrd.iloc[:, 0], 0.1 * (ref_xrd.iloc[:, 2] + 2000), ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$"
    )
    ax.stem(
        pdfcard.iloc[:, 0],
        pdfcard.iloc[:, 2] * 8,
        linefmt=colors[3],
        markerfmt="none",
        bottom=0,
        orientation="vertical",
        label=r"PDF #00-044-0141",
    )
    ax.text(
        0.95,
        0.95,
        r"PDF + Exp.${\mathrm{\alpha\!-\!MnO_2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )
    ax.text(
        0.95,
        0.88,
        r"Exp.ZHS",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[2],
        va="top",
        ha="right",
        fontfamily="Arial",
    )

    ax.set_xlim(5, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=-3))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=-3))

    ax.set_ylim(0, 13000)
    ax.set_ylabel("Relative Intensity (arb.u.)", fontsize=11)
    ax.set(yticks=[])


def plot_contour_map(ax, mapping_spectum, cmap="coolwarm"):
    """绘制等高线图"""
    ax.contourf(
        mapping_spectum.index, mapping_spectum.columns.to_frame()["range"], mapping_spectum.T, cmap=cmap, levels=100
    )
    ax.set_xlim(5, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=-3))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=-3))
    ax.set_yticks([])


def add_colorbar(ax, cmap="coolwarm"):
    """添加颜色条"""
    colorbar = Colorbar(
        ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
        location="right",
        orientation="vertical",
        cmap=cmap,
        ticklocation="right",
        spacing="proportional",
        drawedges=False,
    )
    colorbar.set_ticks([])
    ax.text(1.08, 0.97, "High", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")
    ax.text(1.08, 0.08, "Low", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")


def add_pdf(ax, pdfcard):
    scale_pdf = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
    offst_pdf = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
    XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
    for i in range(pdfcard.shape[1] // 3):
        temp = pdfcard.iloc[:, [3 * i, 3 * i + 2]].dropna(axis="index", how="all")
        for j in range((temp.shape[0])):
            ax.axvline(
                x=temp.iloc[j, 0],
                ymin=offst_pdf[i],
                ymax=(offst_pdf[i] + temp.iloc[j, 1] * scale_pdf[i]),
                lw=1,
                c=XRDcolors[i],
            )


def add_box(subfig):
    ax = subfig.add_subplot()
    ax.set_position([0, -0.25, 0.6, 0.15])
    ax.text(
        0,
        0.05,
        r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
        horizontalalignment="left",
        verticalalignment="bottom",
        transform=ax.transAxes,
        fontsize=10,
        c="blue",
    )
    ax.spines[:].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor("gainsboro")


def plot_reference_peaks(subfig, mapping_spectum, ref_peaks, colors, index, position=0.5):
    """绘制参考峰"""
    for i, peak in enumerate(ref_peaks):
        ax = subfig.add_axes((0 + i * 0.10, position, 0.06, 0.35), zorder=0)
        temp = mapping_spectum.loc[(mapping_spectum.index >= peak[0] - 0.1) & (mapping_spectum.index <= peak[0] + 0.1)]
        ax.contourf(temp.index, temp.columns.to_frame()["range"], temp.T, cmap="coolwarm", levels=100)
        ax.axvline(peak[0], ls="--", lw=1.0, color="k", alpha=0.6)
        ax.set_xlim(temp.index.min(), temp.index.max())
        ax.yaxis.set_ticks([])
        ax.xaxis.set_major_locator(ticker.FixedLocator(peak))
        ax.spines[["top", "right", "left"]].set_visible(False)
        ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)  # colors[i]
        for j in index:
            ax.axhline(y=j + 1, lw=0.6, ls="--")


# 绘图
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

specical_index = [0, 3, 8, 25, 30]
plot_echem_voltage_time(ax, selected_echem, selected, colors, specical_index)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)
plot_current(ax2, selected_echem, colors)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

plot_xrd_spectra(ax, specical_index, spectum_all, selected, colors, ref_xrd, pdf, offset=800)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)
plot_contour_map(ax, mapping_spectum)
add_colorbar(ax)
add_pdf(ax, pdf)

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
add_box(subfig)
opXRD_ref_peak = pdf.iloc[:, 0].apply(lambda x: [x]).to_list()
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]
plot_reference_peaks(
    subfig, mapping_spectum, opXRD_ref_peak[: len(opXRD_ref_peak) // 2], opXRD_colors, specical_index, position=0.0
)
plot_reference_peaks(
    subfig, mapping_spectum, opXRD_ref_peak[len(opXRD_ref_peak) // 2 :], opXRD_colors, specical_index, position=0.45
)

# 保存图像
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

##### Insitu opCellB

In [ ]:
import csv
import os


def extract_wave_and_data(file_path):
    wave_value = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
            if line.startswith("# Date"):
                recored_time = str(line.split()[3])
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recored_time, wave_value, data


def process_files(directory):
    results = []

    for filename in os.listdir(directory):
        if filename.endswith(".xye"):
            file_path = os.path.join(directory, filename)
            recored_time, wave_value, data = extract_wave_and_data(file_path)
            file_id = filename.split(".")[0]  # Extract the last part of the filename
            for row in data:
                results.append([file_id, recored_time, wave_value] + row)

    return results


# 读取 XRD 数据，并清洗
directory = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview\opCellB_fresh"
)
results = process_files(directory)
pd.DataFrame(results).to_csv(
    path.joinpath(path_out, f"{directory.stem}.csv"),
    header=[r"Number", r"Recorded_time", r"WL", r"2THETA", r"Intensity1", r"Intensity2"],
    index=None,
)

In [ ]:
# 读取数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results")
pdf = pd.read_excel(
    path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_0_9535.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Exsitu", r"References", r"References.csv"), sep=",", index_col=None, header=0, comment="#"
)
xrd["sum"] = xrd["Intensity1"] + xrd["Intensity2"]
xrda = xrd.pivot(index="2THETA", columns="Number", values="sum").reset_index(drop=False)
xrda.to_csv(path.joinpath(path_out, "spectrum.csv"), header=True, index=None)

data = pd.read_csv(
    path.joinpath(path_xrd, r"Overview", r"opCellB_fresh", r"opCellB_fresh.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
)
data["sum"] = data["Intensity1"] + data["Intensity2"]
dataa = data.pivot(index="2THETA", columns="Number", values="sum").reset_index(drop=False)
dataa.to_csv(path.joinpath(path_out, "spectrum.csv"), header=True, index=None)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)
labels = [r"ZHS", r"$\mathrm{\alpha MnO_2}$", r"opCellB"]
for i in range(xrda.shape[1] - 1):
    temp = xrda.iloc[:, [0, i + 1]].dropna(axis=0, how="any").iloc[1:, :]
    ax.plot(temp.iloc[:, 0], temp.iloc[:, 1] + 2000 * i, ls="-", lw=1.0, c=colors[i], label=labels[i])
ax.plot(dataa.iloc[:, 0], dataa.iloc[:, 1] + 2500, ls="-", lw=1.0, c=colors[2], label=labels[2])
ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 20 + 2000,
    linefmt=colors[3],
    markerfmt="none",
    bottom=2000,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(10, 46)
ax.set_xlabel(
    "2-Theta (degree), $\mathrm{\lambda = 0.953 \AA}$",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=-2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=-2))

ax.set_ylim(2000, 15000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.75,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinB_3_600.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### Operando XRD, CoinA

##### 谱线上的时间

In [ ]:
# 读取文件中时间戳
filelist = list(
    path(
        r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview\opCellA"
    ).glob(r"*.xye")
)
range_index, wave_length, time_processed = [], [], []

for path_file in filelist:
    with open(path_file, "r") as file:
        lines = file.readlines()
        file_id = path_file.stem.split("-")[-1].split(".")[0][-5:]
        range_index.append(file_id)

        # 解析文件内容
        for line in lines:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
                wave_length.append(wave_value)
            elif line.startswith("# Date"):
                recored_time = str(line.split()[3])
                time_processed.append(recored_time)

df = pd.DataFrame({
    "Range_Index": range_index,
    "Time": time_processed,
    "Wave_Length": wave_length,
})
df["Time"] = pd.to_datetime(df["Time"], format=r"%Y-%m-%d_%H:%M:%S")
df["Range_Index"] = pd.to_numeric(df["Range_Index"])
df["Wave_Length"] = pd.to_numeric(df["Wave_Length"])
df.to_csv(
    path.joinpath(path_out, "Time_index_spectrum.csv"),
    sep=",",
    header=True,
    columns=["Range_Index", "Time", "Wave_Length"],
    index=False,
)

##### 匹配谱线和电化学上的时间

In [ ]:
# 电化学上的时间
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview"
)
echem_file = path.joinpath(path_file, r"Echem", r"001-LC-Zn-05MZnSO4-02MMnSO4-alphaMnO2-59mg_02_GCPL_C01.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"opCellA", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum = [np.abs(echem_time - t).argmin() for t in time_spectrum["Time"].values]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
# # index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage")

selected_times = echem["time/s"].iloc[index_spectrum]
selected_voltages = echem["Ewe/V"].iloc[index_spectrum]
ax.scatter(selected_times, selected_voltages, c=colors[1])

# 添加索引文本标注
row_index = index_voltage["index"].iloc[index_voltage.index.isin([0, 3, 8, 25, 30])]
for i, idx in enumerate(row_index):
    ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], marker="o")
    ax.text(
        echem["time/s"].iloc[idx],
        echem["Ewe/V"].iloc[idx] + 0.03,
        str(row_index.index[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max())
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax2 = ax.twinx()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", labelsize=9, right=True, labelright=True)

ax.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)
ax2.legend(loc="upper left", bbox_to_anchor=(0.8, 1), frameon=False, fontsize=11)

plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

##### Overview

In [ ]:
# 读取 alfaMnO2capillary 数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results")
ref_xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Exsitu", r"References", r"spectrum.csv"), sep=",", index_col=None, header=0, comment="#"
)
# 读取 PDF card 的数据
pdf = pd.read_excel(
    path.joinpath(path_xrd, r"PDFconvert", r"MnO2_00-040-0141_WL_0_9535.xlsx"),
    sheet_name=0,
    index_col=None,
    skiprows=1,
    header=0,
    engine="openpyxl",
    usecols=[0, 1, 2],
    comment="#",
).dropna(axis=0, how="all", ignore_index=True)
pdf = pdf[(pdf["Intensity"] >= 10) & (pdf["2θ"] <= 45)]

In [ ]:
from pathlib import Path

import pandas as pd


def extract_wave_and_data(file_path):
    wave_value = None
    recored_time = None
    data = []

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith("# Wave"):
                wave_value = float(line.split()[3])
            elif line.startswith("# Date"):
                recored_time = line.split()[3]
            elif not line.startswith("#"):
                data.append([float(x) for x in line.split()])

    return recored_time, wave_value, data


def process_files(directory):
    results = []

    for file_path in Path(directory).glob("*.xye"):
        recored_time, wave_value, data = extract_wave_and_data(file_path)
        file_id = file_path.stem.split("_")[-1][-5:]  # Extract the last part of the filename
        for row in data:
            results.append([float(file_id), recored_time, wave_value] + row)

    return results


# 读取 XRD 数据，并清洗
directory = Path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview\opCellA"
)
spectum_all = process_files(directory)

# 转换为 DataFrame 并进行数据处理
spectum_all = pd.DataFrame(
    spectum_all, columns=["range", "recored_time", "wave_value", "2THETA", "Intensity1", "Intensity2"]
)
spectum_all["Cnt2_D1"] = spectum_all["Intensity1"] + spectum_all["Intensity2"]
spectum_all = spectum_all[["range", "2THETA", "Cnt2_D1"]]

In [ ]:
spectum_all.drop(spectum_all[spectum_all["range"] == 28].index, axis=0, inplace=True)

In [ ]:
# 电化学上的时间
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2024-NOTOs\Results\Overview"
)
echem_file = path.joinpath(path_file, r"Echem", r"001-LC-Zn-05MZnSO4-02MMnSO4-alphaMnO2-59mg_02_GCPL_C01.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"opCellA", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum = [np.abs(echem_time - t).argmin() for t in time_spectrum["Time"].values]

# 提取对应的电位和电流数据，并按电位排序
index_voltage = (
    echem.loc[index_spectrum, ["Ewe/V", "<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [0, 3, 5, 8, 10, 12, 14, 17, 21, 25, 30]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all.iloc[:, 0] <= selected["level_0"].iloc[-1] + 1].pivot(
    index=["2THETA"], columns=["range"], values=["Cnt2_D1"]
)

In [ ]:
def plot_echem_voltage_time(ax, selected_echem, selected, colors, index):
    """绘制电化学，电压-时间图"""
    ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoinA")
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
    )
    # 特殊位置颜色标定
    for idx in index:
        temp = selected["index"][selected["level_0"] == idx]
        ax.scatter(selected_echem["Ewe/V"].iloc[temp], selected_echem["time/s"].iloc[temp], c=colors[0], marker="o")

    # 添加索引文本标注
    for i in range(selected.shape[0]):
        temp = selected["index"].iloc[i]
        ax.text(
            selected_echem["Ewe/V"].iloc[temp] + 0.15,
            selected_echem["time/s"].iloc[temp],
            str(selected["level_0"].iloc[i]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

    ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
    ax.set_xlim(0.8, 2.0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

    # 确保时间刻度从数据最开始时间显示
    ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
    ax.set_ylim(
        selected_echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max() + pd.Timedelta(minutes=20)
    )
    ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
    ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 2)))
    ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))
    plt.yticks(rotation=0, horizontalalignment="right")

    ax.text(
        -0.2,
        -0.06,
        r"$\mathrm{1.18 mg\ cm^{-2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )
    ax.text(
        -0.15,
        0,
        r"C/2, 1C=300 mA/g",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )


def plot_current(ax2, selected_echem, colors):
    ax2.plot(selected_echem["<I>/mA"], selected_echem["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")
    ax2.set_xlabel(
        r"Current (mA)",
        fontsize=11,
    )
    ax2.set_xlim(-0.1, 0.1)
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
    ax2.tick_params(axis="both", which="both", labelsize=9, top=True, labeltop=True)
    # ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)


def plot_xrd_spectra(ax, index, spectum_all, selected, colors, ref_xrd, pdfcard, offset=300):
    """绘制 XRD 光谱图"""
    xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
    for i in range(selected.shape[0]):
        ax.plot(
            spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
            spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + offset * (i + 1),
            ls="-",
            lw=1.0,
            c=xrd_colors[i],
            label=str(selected["level_0"].iloc[i]),
        )

    #   标样和 pdf 卡片
    ax.plot(ref_xrd.iloc[:, 0], 0.1 * (ref_xrd.iloc[:, 1] + 4000), ls="-", lw=1.0, c=colors[2], label=r"ZHS")
    ax.plot(
        ref_xrd.iloc[:, 0], 0.1 * (ref_xrd.iloc[:, 2] + 2000), ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$"
    )
    ax.stem(
        pdfcard.iloc[:, 0],
        pdfcard.iloc[:, 2] * 8,
        linefmt=colors[3],
        markerfmt="none",
        bottom=0,
        orientation="vertical",
        label=r"PDF #00-044-0141",
    )
    ax.text(
        0.95,
        0.95,
        r"PDF + Exp.${\mathrm{\alpha\!-\!MnO_2}}$",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[3],
        va="top",
        ha="right",
        fontfamily="Arial",
    )
    ax.text(
        0.95,
        0.88,
        r"Exp.ZHS",
        transform=ax.transAxes,
        fontsize=9,
        color=colors[2],
        va="top",
        ha="right",
        fontfamily="Arial",
    )

    ax.set_xlim(5, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=-3))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=-3))

    ax.set_ylim(0, 12000)
    ax.set_ylabel("Relative Intensity (arb.u.)", fontsize=11)
    ax.set(yticks=[])


def plot_contour_map(ax, mapping_spectum, cmap="coolwarm"):
    """绘制等高线图"""
    ax.contourf(
        mapping_spectum.index, mapping_spectum.columns.to_frame()["range"], mapping_spectum.T, cmap=cmap, levels=100
    )
    ax.set_xlim(5, 45)
    ax.set_xlabel("2-Theta (degree)", fontsize=11)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=-3))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=-3))
    ax.set_yticks([])


def add_colorbar(ax, cmap="coolwarm"):
    """添加颜色条"""
    colorbar = Colorbar(
        ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
        location="right",
        orientation="vertical",
        cmap=cmap,
        ticklocation="right",
        spacing="proportional",
        drawedges=False,
    )
    colorbar.set_ticks([])
    ax.text(1.08, 0.97, "High", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")
    ax.text(1.08, 0.08, "Low", transform=ax.transAxes, fontsize=10, va="top", ha="center", fontfamily="Arial")


def add_pdf(ax, pdfcard):
    scale_pdf = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
    offst_pdf = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
    XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
    for i in range(pdfcard.shape[1] // 3):
        temp = pdfcard.iloc[:, [3 * i, 3 * i + 2]].dropna(axis="index", how="all")
        for j in range((temp.shape[0])):
            ax.axvline(
                x=temp.iloc[j, 0],
                ymin=offst_pdf[i],
                ymax=(offst_pdf[i] + temp.iloc[j, 1] * scale_pdf[i]),
                lw=1,
                c=XRDcolors[i],
            )


def add_box(subfig):
    ax = subfig.add_subplot()
    ax.set_position([0, -0.25, 0.6, 0.15])
    ax.text(
        0,
        0.05,
        r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
        horizontalalignment="left",
        verticalalignment="bottom",
        transform=ax.transAxes,
        fontsize=10,
        c="blue",
    )
    ax.spines[:].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor("gainsboro")


def plot_reference_peaks(subfig, mapping_spectum, ref_peaks, colors, index, position=0.5):
    """绘制参考峰"""
    for i, peak in enumerate(ref_peaks):
        ax = subfig.add_axes((0 + i * 0.10, position, 0.06, 0.35), zorder=0)
        temp = mapping_spectum.loc[(mapping_spectum.index >= peak[0] - 0.1) & (mapping_spectum.index <= peak[0] + 0.1)]
        ax.contourf(temp.index, temp.columns.to_frame()["range"], temp.T, cmap="coolwarm", levels=100)
        ax.axvline(peak[0], ls="--", lw=1.0, color="k", alpha=0.6)
        ax.set_xlim(temp.index.min(), temp.index.max())
        ax.yaxis.set_ticks([])
        ax.xaxis.set_major_locator(ticker.FixedLocator(peak))
        ax.spines[["top", "right", "left"]].set_visible(False)
        ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)  # colors[i]
        for j in index:
            ax.axhline(y=j + 1, lw=0.6, ls="--")


# 绘图
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

specical_index = [0, 3, 8, 25, 30]
plot_echem_voltage_time(ax, selected_echem, selected, colors, specical_index)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)
plot_current(ax2, selected_echem, colors)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

plot_xrd_spectra(ax, specical_index, spectum_all, selected, colors, ref_xrd, pdf, offset=800)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)
plot_contour_map(ax, mapping_spectum)
add_colorbar(ax)
add_pdf(ax, pdf)

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
add_box(subfig)
opXRD_ref_peak = pdf.iloc[:, 0].apply(lambda x: [x]).to_list()
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]
plot_reference_peaks(
    subfig, mapping_spectum, opXRD_ref_peak[: len(opXRD_ref_peak) // 2], opXRD_colors, specical_index, position=0.0
)
plot_reference_peaks(
    subfig, mapping_spectum, opXRD_ref_peak[len(opXRD_ref_peak) // 2 :], opXRD_colors, specical_index, position=0.45
)

# 保存图像
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_NOTOs_coinA_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

### Operando XRD，Coin2

#### Echem 和 谱线之间的匹配

In [ ]:
# 读取 master 文件中时间戳
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin2\Overivew"
)
with open(path.joinpath(path_file, r"operando2-alfaMnO2-1704mg-c5.uxd"), "r") as file:
    lines = file.readlines()
date_measured, range_index, time_processed = None, [], []
# 解析文件内容
for line in lines:
    if line.startswith("_DATEMEASURED"):
        date_measured = line.split("=")[1].strip()
    elif line.startswith("; Data for range"):
        range_number = line.split(" ")[4].strip()
        range_index.append(range_number)
    elif line.startswith("_TIMESTARTED"):
        time_processed.append(line.split("=")[1].strip())
df = pd.DataFrame({"Date_Measured": [date_measured] * len(time_processed), "Time_Processed": time_processed})
df["Date_Measured"] = pd.to_datetime(
    df["Date_Measured"],
)
df["Time_Stamp(s)"] = pd.to_numeric(df["Time_Processed"])
df["Time_Processed"] = pd.to_timedelta(df["Time_Stamp(s)"], unit="s")
df["Time"] = df["Date_Measured"] + df["Time_Processed"]
df.to_csv(
    path.joinpath(path_out, "Time_index_spectrum.csv"),
    sep=",",
    header=True,
    columns=["Time", "Time_Stamp(s)"],
    index=True,
)

#### 匹配谱线和电化学上的时间

In [ ]:
# 电化学上的时间
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin2\Overivew"
)
with open(path.joinpath(path_file, r"EchemOperando2_c5_1704mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    path.joinpath(path_file, r"EchemOperando2_c5_1704mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage")
ax.scatter(echem["time/s"].iloc[index_spectrum], echem["Ewe/V"].iloc[index_spectrum])

# 添加索引文本标注
row_index = index_voltage["index"].iloc[index_voltage.index.isin([0, 7, 15, 26, 39, 61, 88, 112, 176])]
for i, idx in enumerate(row_index):
    ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], marker="o")
    ax.text(
        echem["time/s"].iloc[idx],
        echem["Ewe/V"].iloc[idx] + 0.03,
        str(row_index.index[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 2.05)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠
ax.legend(loc="upper left", bbox_to_anchor=(0.0, 1.05), ncols=1, frameon=False, labelcolor="linecolor", fontsize=11)

ax2 = ax.twinx()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=False,
    left=False,
    right=True,
    labelbottom=False,
    labelleft=False,
    labeltop=False,
    labelright=True,
)
ax2.legend(loc="upper left", bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor="linecolor", fontsize=11)

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### Overview

In [ ]:
# 读取 alfaMnO2capillary 数据
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)
# 读取 PDF card 的数据
pdfMo = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

In [ ]:
# 分割 operando 的数据，并写入 csv 数据
from io import StringIO

data_blocks = []
current_range = None
capturing_data = False
data_lines = []
path_file = path(
    r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin2\Overivew"
)
with open(path.joinpath(path_file, r"operando2-alfaMnO2-1704mg-c5.uxd"), "r") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("; Data for range"):
            # 处理之前的数据块
            if current_range is not None and data_lines:
                df = pd.read_csv(StringIO("\n".join(data_lines)), sep="\s+", header=None, names=["2THETA", "Cnt2_D1"])
                df["range"] = current_range
                data_blocks.append(df)
                data_lines = []
                capturing_data = False
            # 提取当前range的数值
            parts = stripped_line.split()
            current_range = int(parts[-1])
        elif stripped_line.startswith("; 2THETA"):
            capturing_data = True  # 标记开始读取数据行
        elif capturing_data:
            if not stripped_line or stripped_line.startswith(";"):
                capturing_data = False  # 空行或注释行结束数据块
            else:
                data_lines.append(stripped_line)
    # 处理最后一个数据块
    if current_range is not None and data_lines:
        df = pd.read_csv(StringIO("\n".join(data_lines)), sep="\s+", header=None, names=["2THETA", "Cnt2_D1"])
        df["range"] = current_range
        data_blocks.append(df)

# 合并所有数据
spectum_all = pd.concat(data_blocks, ignore_index=True)
spectum_all.to_csv(path.joinpath(path_out, "Spectum_all.csv"), sep=",", header=True, index=False)

# # 查看前几行数据
# print(spectum_all.head(10))

In [ ]:
# 电化学上的时间
with open(path.joinpath(path_file, r"EchemOperando2_c5_1704mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    path.joinpath(path_file, r"EchemOperando2_c5_1704mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_file, r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index_voltage.reset_index(drop=False, inplace=True)
selected = (
    index_voltage[
        index_voltage["level_0"].isin([
            0,
            4,
            7,
            10,
            15,
            17,
            19,
            23,
            26,
            31,
            39,
            44,
            49,
            55,
            61,
            67,
            75,
            80,
            88,
            94,
            100,
            106,
            112,
        ])
    ]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all["range"] <= selected["level_0"].iloc[-1] + 1].pivot(
    index="2THETA", columns="range", values="Cnt2_D1"
)

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 图 A
# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
index = [0, 7, 15, 26, 39, 61, 88, 112]
for i, idx in enumerate(index):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        selected_echem["time/s"].iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.408 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 8500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
# ax = subfig.add_subplot()
# ax.set_position([-0.2, 0, 1.0, 1.0])

opXRD_ref_peak = [
    [5.369],
    [6.454],
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.0118],
    [12.2099],
    [12.611],
    [12.755],
    [13.060],
    [14.325],
    [14.918],
    [15.055],
    [15.300],
    [15.452],
    [15.6563],
    [15.802],
    [15.914],
    [16.103],
    [16.2506],
    [16.316],
    [16.541],
    [16.939],
    [17.585],
    [18.457],
    [18.944],
    [19.3207],
    [19.4197],
    [19.598],
    [19.7763],
    [19.8357],
]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 图 A
# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
index = [0, 7, 15, 26, 39, 61, 88, 112]
for i, idx in enumerate(index):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        selected_echem["time/s"].iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.408 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 8500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.68, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_3_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_3_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_3_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
index = [0, 7, 15, 26, 39, 61, 88, 112]
for i, idx in enumerate(index):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        selected_echem["time/s"].iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.408 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 8500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.52,
    0.55,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.15,
    0.03,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.65, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]
# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_4_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_4_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_4_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### 1stDischarge

In [ ]:
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"1stDischarge", "Spectum_all.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
)

# 电化学上的时间
with open(path.joinpath(path_xrd, r"Coin2", r"1stDischarge", r"EchemOperando2_c5_1704mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"1stDischarge", r"EchemOperando2_c5_1704mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"1stDischarge", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
selected = (
    index_voltage[index_voltage["level_0"].isin([0, 4, 7])]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all["range"] <= selected["level_0"].iloc[-1] + 1].pivot(
    index="2THETA", columns="range", values="Cnt2_D1"
)
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(selected_echem["Ewe/V"].iloc[selected["index"]], FD1st_time.iloc[selected["index"]], c=colors[1])

# 特殊位置颜色标定
index = [0, 7]
for i, idx in enumerate(index):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        FD1st_time.iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.02,
        FD1st_time.iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 4)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=0))

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 1500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.65, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(selected_echem["Ewe/V"].iloc[selected["index"]], FD1st_time.iloc[selected["index"]], c=colors[1])

# 特殊位置颜色标定
index = [0, 7]
for i, idx in enumerate(index):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        FD1st_time.iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.02,
        FD1st_time.iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 4)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=0))

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 1500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.70, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_FD1st_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### Consequent Cycle

In [ ]:
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"ConsequentCycle", "Spectum_all.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
)

# 电化学上的时间
with open(path.joinpath(path_xrd, r"Coin2", r"ConsequentCycle", r"EchemOperando2_c5_1704mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"ConsequentCycle", r"EchemOperando2_c5_1704mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"ConsequentCycle", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
selected = (
    index_voltage[index_voltage["level_0"].isin([7, 10, 13, 15, 16, 19, 23, 26, 30, 34, 36, 39])]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[(echem.index <= selected["index"].iloc[-1]) & (echem.index >= selected["index"].iloc[0])]
mapping_spectum = spectum_all[
    (spectum_all["range"] <= selected["level_0"].iloc[-1] + 1)
    & (spectum_all["range"] >= selected["level_0"].iloc[0] + 1)
].pivot(index="2THETA", columns="range", values="Cnt2_D1")
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [7, 15, 26, 39]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 18)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=0))

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.65, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [7, 15, 26, 39]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 18)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4500)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.70, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### 第二个循环

In [ ]:
# 选择需要的电化学数据以及对应的谱线
selected = (
    index_voltage[index_voltage["level_0"].isin([39, 44, 49, 55, 61, 67, 75, 80, 88, 94, 100, 106, 112])]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[(echem.index <= selected["index"].iloc[-1]) & (echem.index >= selected["index"].iloc[0])]
mapping_spectum = spectum_all[
    (spectum_all["range"] <= selected["level_0"].iloc[-1] + 1)
    & (spectum_all["range"] >= selected["level_0"].iloc[0] + 1)
].pivot(index="2THETA", columns="range", values="Cnt2_D1")
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [39, 61, 88, 112]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 42)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 5000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.70, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_3_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_3_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_3_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [39, 61, 88, 112]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 42)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.408 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 5000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.65, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_4_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_4_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin1_ConsequentCycle_4_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

#### 失效分析

In [ ]:
path_xrd = path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"Failure", "Spectum_all.csv"), sep=",", index_col=None, header=0, comment="#"
)

# 电化学上的时间
with open(path.joinpath(path_xrd, r"Coin2", r"Failure", r"EchemOperando2_c5_1704mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"Failure", r"EchemOperando2_c5_1704mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    path.joinpath(path_xrd, r"Coin2", r"Failure", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
selected = (
    index_voltage[index_voltage["level_0"].isin([112, 120, 130, 140, 150, 160, 176])]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[(echem.index <= selected["index"].iloc[-1]) & (echem.index >= selected["index"].iloc[0])]
mapping_spectum = spectum_all[
    (spectum_all["range"] <= selected["level_0"].iloc[-1] + 1)
    & (spectum_all["range"] >= selected["level_0"].iloc[0] + 1)
].pivot(index="2THETA", columns="range", values="Cnt2_D1")
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [112, 176]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 35)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.65, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_1_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
index = [112, 176]
for idx in index:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 35)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=0))

ax2 = ax.twiny()
ax2.set_position([0, 0, 1, 1])
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = selected["level_0"].isin(index).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.4, -0.25, 1.2, 1.2])
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes([1.03, 0.1, 0.08, 0.8]),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, -0.25, 0.6, 0.15])

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position([-0.27, -0.125, 0.70, 0.08])

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],
    [9.775],
    [11.194],
    [11.591],
    [11.837],
    [11.945],
    [12.611],
    [12.755],
    [13.060],
    [14.918],
    [15.055],
    [15.276],
    [15.452],
    [15.802],
    [15.914],
    [16.103],
    [16.541],
    [17.585],
    [18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    path.joinpath(path_out, r"opXRD_coin2_Failure_2_600_echem.png"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
)
plt.show()

### Operando XRD, Coin1

#### Ex situ, Mo target XRD 

##### alfaMnO2capillary

In [ ]:
# 读取数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)

line2d = ax.plot(xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[0], label=r"$\alpha$-MnO${_2}$")
ax.legend(
    handles=line2d,
    loc="upper right",
    bbox_to_anchor=(0.98, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.2,
    linefmt=colors[3],
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(5, 35)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2.5, offset=-5))

ax.set_ylim(0, 900)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.90,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_alfaMnO2capillary_300.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_alfaMnO2capillary_600.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### selfstanding electrode

In [ ]:
# 读取数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"selfstandingelectrode.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=r",", index_col=None, header=0, comment="#"
)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)

line2d = ax.plot(xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[0], label=r"$\alpha$-MnO${_2}$")
ax.legend(
    handles=line2d,
    loc="upper right",
    bbox_to_anchor=(0.98, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.2,
    linefmt=colors[3],
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

ax.set_ylim(0, 400)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.95,
    0.90,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_selfstandingelectrode_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_selfstandingelectrode_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 1stCharged_1_624mg electrode

In [ ]:
# 读取数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"1stCharged_1_624mg.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)
xrd2 = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"selfstandingelectrode.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=r",", index_col=None, header=0, comment="#"
)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)

line2d = ax.plot(xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[0], label=r"$\mathrm{1^{st}\ Charged}$")
line2d2 = ax.plot(xrd2.iloc[:, 0], xrd2.iloc[:, 1] + 60, ls="-", lw=1.0, c=colors[1], label=r"$\alpha$-MnO${_2}$")
ax.legend(loc="upper right", bbox_to_anchor=(0.98, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.2,
    linefmt=colors[3],
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

ax.set_ylim(0, 600)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.94,
    0.8,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_1stCharged_1_624mg_300.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_1stCharged_1_624mg_600.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### testcoincell2

In [ ]:
# 读取数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"testcoincell2.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)
xrd2 = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"selfstandingelectrode.uxd"), sep="\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)

In [ ]:
# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_axes((0, 0, 1, 1), zorder=0)
ax.set_box_aspect(0.8)

line2d = ax.plot(xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[0], label=r"Test Coincell2")
line2d2 = ax.plot(xrd2.iloc[:, 0], xrd2.iloc[:, 1] + 60, ls="-", lw=1.0, c=colors[1], label=r"$\alpha$-MnO${_2}$")
ax.legend(loc="upper right", bbox_to_anchor=(0.4, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.2,
    linefmt=colors[3],
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)

ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

ax.set_ylim(0, 800)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

ax.text(
    0.34,
    0.8,
    r"PDF#00-044-0141",
    transform=ax.transAxes,
    fontsize=8,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
    fontweight="bold",
    zorder=6,
)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_testcoincell2_300.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_testcoincell2_600.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### Echem 和 谱线之间的匹配

In [ ]:
# 读取 master 文件中时间戳
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin1\Overivew"
)
with open(Path.joinpath(path_file, r"operando-alfaMnO2-1544mg-c10.uxd"), "r") as file:
    lines = file.readlines()
date_measured, range_index, time_processed = None, [], []
# 解析文件内容
for line in lines:
    if line.startswith("_DATEMEASURED"):
        date_measured = line.split("=")[1].strip()
    elif line.startswith("; Data for range"):
        range_number = line.split(" ")[4].strip()
        range_index.append(range_number)
    elif line.startswith("_TIMESTARTED"):
        time_processed.append(line.split("=")[1].strip())
df = pd.DataFrame({"Date_Measured": [date_measured] * len(time_processed), "Time_Processed": time_processed})
df["Date_Measured"] = pd.to_datetime(
    df["Date_Measured"],
)
df["Time_Stamp(s)"] = pd.to_numeric(df["Time_Processed"])
df["Time_Processed"] = pd.to_timedelta(df["Time_Stamp(s)"], unit="s")
df["Time"] = df["Date_Measured"] + df["Time_Processed"]
df.to_csv(
    Path.joinpath(path_out, "Time_index_spectrum.csv"),
    sep=",",
    header=True,
    columns=["Time", "Time_Stamp(s)"],
    index=True,
)

#### 匹配谱线和电化学上的时间

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin1\Overivew"
)
with open(Path.joinpath(path_file, r"EchemOperando1_c10_1544mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    Path.joinpath(path_file, r"EchemOperando1_c10_1544mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()  # noqa: ERA001

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
# index_voltage.reset_index(drop=False, inplace=True)  # noqa: ERA001

In [ ]:
%matplotlib inline

# 画图
# gridspec inside gridspec
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage")
ax.scatter(echem["time/s"].iloc[index_spectrum], echem["Ewe/V"].iloc[index_spectrum])

# 添加索引文本标注
row_index = index_voltage["index"].iloc[index_voltage.index.isin([0, 17, 37, 54, 87])]
for i, idx in enumerate(row_index):
    ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], marker="o")
    ax.text(
        echem["time/s"].iloc[idx],
        echem["Ewe/V"].iloc[idx] + 0.03,
        str(row_index.index[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(dates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.xaxis.set_major_locator(dates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.xaxis.set_minor_locator(dates.HourLocator(interval=1))

plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠
ax.legend(loc="upper left", bbox_to_anchor=(0, 1.05), ncols=1, frameon=False, labelcolor="linecolor", fontsize=11)
ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=False,
    left=False,
    right=True,
    labelbottom=False,
    labelleft=False,
    labeltop=False,
    labelright=True,
)
ax2.legend(loc="upper left", bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor="linecolor", fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_1_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_1_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### Overview

In [ ]:
# 读取 alfaMnO2capillary 数据
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=",", index_col=None, header=0, comment="#"
)
# 读取 PDF card 的数据
pdfMo = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

In [ ]:
# 分割 operando 的数据，并写入 csv 数据
from io import StringIO

data_blocks = []
current_range = None
capturing_data = False
data_lines = []
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results\Coin1\Overivew"
)
with open(Path.joinpath(path_file, r"operando-alfaMnO2-1544mg-c10.uxd"), "r") as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith("; Data for range"):
            # 处理之前的数据块
            if current_range is not None and data_lines:
                df = pd.read_csv(StringIO("\n".join(data_lines)), sep=r"\s+", header=None, names=["2THETA", "Cnt2_D1"])
                df["range"] = current_range
                data_blocks.append(df)
                data_lines = []
                capturing_data = False
            # 提取当前range的数值
            parts = stripped_line.split()
            current_range = int(parts[-1])
        elif stripped_line.startswith("; 2THETA"):
            capturing_data = True  # 标记开始读取数据行
        elif capturing_data:
            if not stripped_line or stripped_line.startswith(";"):
                capturing_data = False  # 空行或注释行结束数据块
            else:
                data_lines.append(stripped_line)
    # 处理最后一个数据块
    if current_range is not None and data_lines:
        df = pd.read_csv(StringIO("\n".join(data_lines)), sep="\s+", header=None, names=["2THETA", "Cnt2_D1"])
        df["range"] = current_range
        data_blocks.append(df)

# 合并所有数据
spectum_all = pd.concat(data_blocks, ignore_index=True)
spectum_all.to_csv(Path.joinpath(path_out, "Spectum_all.csv"), sep=",", header=True, index=False)

# # 查看前几行数据
# print(spectum_all.head(10))

In [ ]:
# 电化学上的时间
with open(Path.joinpath(path_file, r"EchemOperando1_c10_1544mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    Path.joinpath(path_file, r"EchemOperando1_c10_1544mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
selected = (
    index_voltage[index_voltage["level_0"].isin([0, 5, 10, 17, 22, 27, 32, 37, 42, 48, 54])]
    .sort_values(by="level_0", ascending=True)
    .reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all["range"] <= selected["level_0"].iloc[-1] + 1].pivot(
    index="2THETA", columns="range", values="Cnt2_D1"
)

In [ ]:
%matplotlib inline

# 图 A
# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
index = [0, 17, 37, 54]
for idx in index:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[
            selected["index"][selected["level_0"] == idx]
        ],
        selected_echem["time/s"].iloc[
            selected["index"][selected["level_0"] == idx]
        ],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(dates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(dates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(dates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.288 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0],colors[1],colors[1],colors[0],colors[1],colors[1],colors[1],colors[0],colors[1],colors[1],colors[0],]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4800)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1]* pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
# ax = subfig.add_subplot()
# ax.set_position([-0.2, 0, 1.0, 1.0])

opXRD_ref_peak = [
    [5.369],[6.454],[7.451],[9.775],[11.194],[11.591],
    [11.837],[11.945],[12.0118],[12.2099],[12.611],[12.755],
    [13.060],[14.325],[14.918],[15.055],[15.300],[15.452],[15.6563],
    [15.802],[15.914],[16.103],[16.2506],[16.316],[16.541],[16.939],
    [17.585],[18.457],[18.944],[19.3207],[19.4197],[19.598],[19.7763],[19.8357],
    ]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(axis="both", which="both", labelcolor="k", bottom=True, labelbottom=True)
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.dates as mdates

# 图 A
# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
for i, idx in enumerate([0, 17, 37, 54]):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        selected_echem["time/s"].iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.288 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [
    colors[0],colors[1],colors[1],colors[0],colors[1],
    colors[1],colors[1],colors[0],colors[1],colors[1],colors[0],
]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4800)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.68, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],[16.103],
    [16.541],[17.585],[18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_3_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_3_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], selected_echem["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(
    selected_echem["Ewe/V"].iloc[selected["index"]], selected_echem["time/s"].iloc[selected["index"]], c=colors[1]
)

# 特殊位置颜色标定
for i, idx in enumerate([0, 17, 37, 54]):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        selected_echem["time/s"].iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.15,
        selected_echem["time/s"].iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_xlim(0.85, 1.9)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%h-%d, %H:%m"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=[6 * i for i in range(4)]))
ax.yaxis.set_minor_locator(mdates.HourLocator(interval=1))

plt.yticks(rotation=0, horizontalalignment="right")  # 旋转刻度标签，防止重叠
# ax.legend(loc='upper right', bbox_to_anchor=(-0.1, -0.1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
ax.text(
    -0.1,
    -0.06,
    r"$\mathrm{3.288 mg\ cm^{-2}}$",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    -0.1,
    0,
    r"C/10, 1C=250 mA/g",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [
    colors[0],colors[1],colors[1],colors[0],colors[1],colors[1],colors[1],
    colors[0],colors[1],colors[1],colors[0],
]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4800)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])

ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.52,
    0.55,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.15,
    0.03,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.65, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],
    [16.103],[16.541],[17.585],[18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in [0, 17, 27, 37, 48, 54]:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_4_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_4_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 1stDischarge

In [ ]:
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=r",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stDischarge", "Spectum_all.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
)

# 电化学上的时间
with open(Path.joinpath(path_xrd, r"Coin1", r"1stDischarge", r"EchemOperando1_c10_1544mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stDischarge", r"EchemOperando1_c10_1544mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stDischarge", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [0, 5, 10, 14, 17]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[echem.index <= selected["index"].iloc[-1]]
mapping_spectum = spectum_all[spectum_all["range"] <= selected["level_0"].iloc[-1] + 1].pivot(
    index="2THETA", columns="range", values="Cnt2_D1"
)
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(selected_echem["Ewe/V"].iloc[selected["index"]], FD1st_time.iloc[selected["index"]], c=colors[1])

# 特殊位置颜色标定
for i, idx in enumerate([index[0], index[-1]]):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        FD1st_time.iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.02,
        FD1st_time.iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 15)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[1], colors[1], colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.65, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],[16.103],
    [16.541],[17.585],[18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FD1st_1_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FD1st_1_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
ax.scatter(selected_echem["Ewe/V"].iloc[selected["index"]], FD1st_time.iloc[selected["index"]], c=colors[1])

# 特殊位置颜色标定
for i, idx in enumerate([index[0], index[-1]]):
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected["index"][selected["level_0"] == idx]],
        FD1st_time.iloc[selected["index"][selected["level_0"] == idx]],
        c=colors[0],
        marker="o",
    )

# 添加索引文本标注
for i in range(selected.shape[0]):
    ax.text(
        selected_echem["Ewe/V"].iloc[selected["index"].iloc[i]] + 0.02,
        FD1st_time.iloc[selected["index"].iloc[i]],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.45)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 15)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[1], colors[1], colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3000)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.70, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],[16.103],
    [16.541],[17.585],[18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FD1st_2_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FD1st_2_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 1stCharge + 2ndDischarge

In [ ]:
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=r",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stCharge+2ndDischarge", "Spectum_all.csv"),
    sep=r",",
    index_col=None,
    header=0,
    comment="#",
)

# 电化学上的时间
with open(
    Path.joinpath(path_xrd, r"Coin1", r"1stCharge+2ndDischarge", r"EchemOperando1_c10_1544mg_C02.txt"), "r"
) as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stCharge+2ndDischarge", r"EchemOperando1_c10_1544mg_C02.txt"),
    sep="\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"1stCharge+2ndDischarge", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [17, 22, 27, 32, 37, 40, 45, 50, 54]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[(echem.index <= selected["index"].iloc[-1]) & (echem.index >= selected["index"].iloc[0])]
mapping_spectum = spectum_all[
    (spectum_all["range"] <= selected["level_0"].iloc[-1] + 1)
    & (spectum_all["range"] >= selected["level_0"].iloc[0] + 1)
].pivot(index="2THETA", columns="range", values="Cnt2_D1")
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
for idx in [index[0], index[2], index[4], index[6], index[8]]:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 30)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[0], colors[1], colors[0], colors[1], colors[0], colors[1], colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4100)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.65, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],
    [16.103],[16.541],[17.585],[18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FC1st_FD2nd_1_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FC1st_FD2nd_1_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
for idx in [index[0], index[2], index[4], index[6], index[8]]:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 30)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[0], colors[1], colors[0], colors[1], colors[0], colors[1], colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 4100)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.70, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],
    [12.755],[13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],
    [16.103],[16.541],[17.585],[18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FC1st_FD2nd_2_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_FC1st_FD2nd_2_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 失效分析

In [ ]:
path_xrd = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XRD\Operando\αMnO2\2022-ICMAB\Results")
# 读取 alfaMnO2capillary 数据
xrd = pd.read_csv(
    Path.joinpath(path_xrd, r"Ex situ", r"alfaMnO2capillary.uxd"), sep=r"\s+", index_col=None, header=0, comment="#"
)

# 读取 PDF card 的数据
pdf = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF-00-044-0141-Mo.csv"), sep=r",", index_col=None, header=0, comment="#"
)
pdfMo = pd.read_csv(
    Path.joinpath(path_xrd, r"PDFCards", "PDF_1stDischarge.csv"),
    sep=r",",
    index_col=None,
    header=0,
    skiprows=1,
    comment="#",
)

# 读取 spectrum 的数据
spectum_all = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"Failure", "Spectum_all.csv"), sep=",", index_col=None, header=0, comment="#"
)

# 电化学上的时间
with open(Path.joinpath(path_xrd, r"Coin1", r"Failure", r"EchemOperando1_c10_1544mg_C02.txt"), "r") as file:
    lines = file.readlines()
for line in lines:
    if line.startswith("Nb header lines"):
        line_skip = int(line.split(":")[1].strip())

echem = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"Failure", r"EchemOperando1_c10_1544mg_C02.txt"),
    sep=r"\t",
    index_col=None,
    header=0,
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1, 2],
).dropna(axis=1, how="all", inplace=False)
echem["time/s"] = pd.to_datetime(
    echem["time/s"],
)
echem["Ewe/V"] = pd.to_numeric(
    echem["Ewe/V"],
)
echem["<I>/mA"] = pd.to_numeric(
    echem["<I>/mA"],
)
echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_xrd, r"Coin1", r"Failure", r"Time_index_spectrum.csv"),
    sep=",",
    index_col=0,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
index_spectrum = [abs(echem["time/s"] - t).idxmin() for t in time_spectrum["Time"]]
index_voltage = (
    echem.loc[index_spectrum, [r"Ewe/V", r"<I>/mA"]].reset_index(drop=False).sort_values(by="Ewe/V", ascending=True)
)
index_voltage.reset_index(drop=False, inplace=True)

In [ ]:
# 选择需要的电化学数据以及对应的谱线
index = [54, 59, 64, 69, 74, 79, 84, 87]
selected = (
    index_voltage[index_voltage["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_echem = echem[(echem.index <= selected["index"].iloc[-1]) & (echem.index >= selected["index"].iloc[0])]
mapping_spectum = spectum_all[
    (spectum_all["range"] <= selected["level_0"].iloc[-1] + 1)
    & (spectum_all["range"] >= selected["level_0"].iloc[0] + 1)
].pivot(index="2THETA", columns="range", values="Cnt2_D1")
FD1st_time = (selected_echem["time/s"] - selected_echem["time/s"].iloc[0]).dt.total_seconds() / 3600

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
for idx in [index[0], index[2], index[-1]]:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 30)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position((0.0, 0.0, 1.0, 1.0))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[0], colors[1], colors[1], colors[1], colors[1], colors[0], colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3700)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="blue",
)
ax.text(
    0.48,
    0.55,
    r"Zn(HSO$_{4}$)$_{2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=10, c="k"
)
ax.text(
    0.12,
    0.05,
    r"K$_{0.66}$Mn$_4$O$_{8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=10,
    c="purple",
)
ax.text(
    0.66,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.65, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.66,
    0.55,
    r"$\mathrm{K_{0.66}Mn_4O_8}$ #04-023-7544",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.67,
    0.02,
    r"$\mathrm{Zn(HSO_4)_2}$ #00-052-0258",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],
    [16.103],[16.541],[17.585],[18.457],
]

# KMnO
opXRD_colors = [K1, K1, K2, K1, K1, K1, K1, K1, K3, K2, K4, K1, K2, K2, K1, K1, K3, K4, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_Failure_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_Failure_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline

# gridspec inside gridspec
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 3, 2, 2], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

ax.plot(selected_echem["Ewe/V"], FD1st_time, ls="-", lw=1.0, c=colors[0], label=r"opCoin_1")
for i in selected["index"]:
    ax.scatter(
        selected_echem["Ewe/V"].iloc[selected_echem.index == i], FD1st_time.iloc[FD1st_time.index == i], c=colors[1]
    )

# 特殊位置颜色标定
for idx in [index[0], index[2], index[-1]]:
    row_idx = selected.loc[selected["level_0"] == idx].iloc[0, 1]  # 提取索引值
    ax.scatter(selected_echem.loc[row_idx, "Ewe/V"], FD1st_time.loc[row_idx], c=colors[0], marker="o")

# 添加索引文本标注
for i in range(selected.shape[0]):
    row_idx = selected_echem.index == selected["index"].iloc[i]  # 提取索引值
    ax.text(
        selected_echem.loc[row_idx, "Ewe/V"] + 0.02,
        FD1st_time.iloc[row_idx],
        str(selected["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

ax.set_xlabel(
    r"Voltage (V)",
    fontsize=11,
)
ax.set_xlim(0.85, 1.85)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.05))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.05))

ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(0, 30)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

# ax.legend(loc='upper right', bbox_to_anchor=(0.9, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)
# ax.text(-0.1, -0.06, r'$\mathrm{3.288 mg\ cm^{-2}}$', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)
# ax.text(-0.1, 0, r'C/10, 1C=250 mA/g', transform=ax.transAxes, fontsize=9, color=colors[3], va='top', ha='right', fontfamily='Arial',)

ax2 = ax.twiny()
ax2.set_position((0.0, 0.0, 1.0, 1.0))
ax2.set_box_aspect(2.4)

ax2.plot(selected_echem["<I>/mA"], FD1st_time, ls="--", lw=1.0, c=colors[3], label=r"Current")

ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.2, 0.2)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    bottom=False,
    top=True,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False,
    labeltop=True,
    labelright=False,
)
# ax2.legend(loc='upper left', bbox_to_anchor=(0.15, 1.05), ncols=1, frameon=False, labelcolor='linecolor', fontsize=11)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

line2d = ax.plot(
    xrd.iloc[:, 0], xrd.iloc[:, 1] + 100, ls="-", lw=1.0, c=colors[3], label=r"$\alpha$-MnO${_2}$ capillary"
)
stem = ax.stem(
    pdf.iloc[:, 0],
    pdf.iloc[:, 2] * 1.5,
    linefmt="k",
    markerfmt="none",
    bottom=0,
    orientation="vertical",
    label=r"PDF #00-044-0141",
)
ax.legend(loc="upper right", bbox_to_anchor=(0.45, 1.0), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)

xrd_colors = [colors[0], colors[1], colors[0], colors[1], colors[1], colors[1], colors[1], colors[0],colors[0]]
for i in range(selected.shape[0]):
    ax.plot(
        spectum_all["2THETA"][spectum_all["range"] == selected["level_0"].iloc[i] + 1],
        spectum_all["Cnt2_D1"][spectum_all["range"] == selected["level_0"].iloc[i] + 1] + 300 * i,
        ls="-",
        lw=1.0,
        c=xrd_colors[i],
        label=str(selected["level_0"].iloc[i]),
    )

ax.set_xlim(5, 20)
ax.set_xlabel(
    r"2-Theta$\mathrm{\ _{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))

ax.set_ylim(0, 3700)
ax.set_ylabel(
    "Relative Intensity (arb.u.)",
    fontsize=11,
)
ax.set(yticks=[])
ax.tick_params(axis="both", labelsize=9)

# 图 C
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.25, 1.2, 1.2))
ax.set_box_aspect(1)

ax.contourf(mapping_spectum.index, mapping_spectum.columns, mapping_spectum.T, cmap="coolwarm", levels=100)

ax.set_yticks([])
ax.set_xlim(5, 20)
ax.set_xlabel(
    "2-Theta$\mathrm{_{Mo}}$ (degree)",
    fontsize=11,
)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5, offset=-5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=-5))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((1.03, 0.1, 0.08, 0.8)),
    location="right",
    orientation="vertical",
    cmap="coolwarm",
    ticklocation="right",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.text(
    1.08,
    0.97,
    "High",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)
ax.text(
    1.08,
    0.08,
    "Low",
    transform=ax.transAxes,
    fontsize=10,
    va="top",
    ha="center",
    fontfamily="Arial",
)

pdf_scale = [0.001, 0.006, 0.001, 0.03, 0.002, 0.001, 0.0001, 0.0005]
XRD_y = [0, 0.15, 0.15, 0.15, 0.15, 0.07, 0.0, 0.09]
XRDcolors = ["blue", "k", "k", "k", "k", "orange", "purple", "green"]
for i in range(pdfMo.shape[0]):
    temp = pdfMo.iloc[:, 2 * i : 2 * i + 2].dropna(axis="index", how="all")
    for j in range((temp.shape[0])):
        ax.axvline(
            x=temp.iloc[j, 0], ymin=XRD_y[i], ymax=(XRD_y[i] + temp.iloc[j, 1] * pdf_scale[i]), lw=1, c=XRDcolors[i]
        )

# 图 D
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.25, 0.6, 0.15))

ax.text(
    0,
    0.55,
    r"$\alpha$-MnO$\mathrm{_2}$ #00-044-0141",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="blue",
)
ax.text(
    0.46,
    0.55,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="orange",
)
ax.text(
    0, 0.05, r"ZHS", horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes, fontsize=9, c="k"
)
ax.text(
    0.10,
    0.03,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="purple",
)
ax.text(
    0.69,
    0.05,
    r"Ti #00-044-1294",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c="green",
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [[5.87], [8.31], [13.15], [17.05], [18.19], [18.98]]
opXRD_colors = ["blue", "blue", "blue", "blue", "green", "blue"]

for i in range(len(opXRD_ref_peak)):
    ax = subfig.add_axes((0.0 + i * 0.10, 0, 0.095, 0.76), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i],
        bottom=True,
        top=False,
        left=True,
        right=False,
        labelbottom=True,
        labelleft=True,
        labeltop=False,
        labelright=False,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

# 图 E
subfig = fig.add_subfigure(gs[0:2, 2:4], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.27, -0.125, 0.70, 0.08))

K1 = "#295488"
K2 = "k"
K3 = "purple"
K4 = "orange"

ax.text(
    0,
    0.55,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 5H_2O}$ #00-060-0655, #04-012-8189",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K1,
)
ax.text(
    0,
    0.02,
    r"$\mathrm{Zn_4SO_4(OH)_6 \cdot 3H_2O}$ #01-082-3605, #00-039-0689",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K2,
)
ax.text(
    0.62,
    0.55,
    r"$\mathrm{ZnMn_3O_7 \cdot 3H_2O}$ #04-025-3574",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K3,
)
ax.text(
    0.63,
    0.02,
    r"$\mathrm{ZnSO_4 \cdot 6H_2O}$ #00-001-0352",
    horizontalalignment="left",
    verticalalignment="bottom",
    transform=ax.transAxes,
    fontsize=9,
    c=K4,
)
ax.spines[:].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor("gainsboro")

opXRD_ref_peak = [
    [7.451],[9.775],[11.194],[11.591],[11.837],[11.945],[12.611],[12.755],
    [13.060],[14.918],[15.055],[15.276],[15.452],[15.802],[15.914],
    [16.103],[16.541],[17.585],[18.457],
]

# ZnMnO
opXRD_colors = [K1, K1, K2, K3, K1, K1, K1, K4, K3, K2, K3, K1, K2, K2, K1, K1, K1, K3, K1]

for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.7, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i][0] - 0.1) & (mapping_spectum.index <= opXRD_ref_peak[i][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(axis="both", which="both", labelcolor=opXRD_colors[i], bottom=True, labelbottom=True)
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) // 3 + 1):
    # print(i+len(opXRD_ref_peak)//3+1)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.35, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + len(opXRD_ref_peak) // 3 + 1]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + len(opXRD_ref_peak) // 3 + 1],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")
for i in range(len(opXRD_ref_peak) - 2 * len(opXRD_ref_peak) // 3 - 2):
    # print(i+2*len(opXRD_ref_peak)//3+2)
    ax = subfig.add_axes((-0.25 + i * 0.09, 0.0, 0.08, 0.3), zorder=0)
    temp = mapping_spectum.loc[
        (mapping_spectum.index >= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] - 0.1)
        & (mapping_spectum.index <= opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0] + 0.1)
    ]
    ax.contourf(temp.index, temp.columns, temp.T, cmap="coolwarm", levels=100)
    ax.axvline(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2][0], ls="--", lw=1.0, color="k", alpha=0.6)
    ax.yaxis.set_ticks([])
    ax.set_ylim(temp.columns.min(), temp.columns.max())
    ax.set_xlim(temp.index.min(), temp.index.max())
    ax.xaxis.set_major_locator(ticker.FixedLocator(opXRD_ref_peak[i + 2 * len(opXRD_ref_peak) // 3 + 2]))
    ax.spines[["top", "right", "left"]].set_visible(False)
    # ax.tick_params(axis='both', which='both', labelcolor='k', bottom=True, labelbottom=True)
    ax.tick_params(
        axis="both",
        which="both",
        labelcolor=opXRD_colors[i + 2 * len(opXRD_ref_peak) // 3 + 2],
        bottom=True,
        labelbottom=True,
    )
    for j in index:
        ax.axhline(y=j + 1, lw=0.6, ls="--")

plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_Failure_2_300_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXRD_coin1_Failure_2_600_echem.tif"),
    pad_inches=0.01,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()